In [8]:


from google.colab import files
uploaded = files.upload()



Saving IR-Plag-Dataset (1).zip to IR-Plag-Dataset (1) (1).zip


In [9]:
import zipfile, os

zip_path = "/content/IR-Plag-Dataset (1).zip"
extract_path = "/content/IR-Plag-Dataset (1)"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Confirm extraction
for root, dirs, files in os.walk(extract_path):
    print(root, "->", len(files), "files")

/content/IR-Plag-Dataset (1) -> 0 files
/content/IR-Plag-Dataset (1)/IR-Plag-Dataset -> 0 files
/content/IR-Plag-Dataset (1)/IR-Plag-Dataset/case-02 -> 0 files
/content/IR-Plag-Dataset (1)/IR-Plag-Dataset/case-02/original -> 1 files
/content/IR-Plag-Dataset (1)/IR-Plag-Dataset/case-02/plagiarized -> 0 files
/content/IR-Plag-Dataset (1)/IR-Plag-Dataset/case-02/plagiarized/L1 -> 0 files
/content/IR-Plag-Dataset (1)/IR-Plag-Dataset/case-02/plagiarized/L1/04 -> 1 files
/content/IR-Plag-Dataset (1)/IR-Plag-Dataset/case-02/plagiarized/L1/09 -> 1 files
/content/IR-Plag-Dataset (1)/IR-Plag-Dataset/case-02/plagiarized/L1/02 -> 1 files
/content/IR-Plag-Dataset (1)/IR-Plag-Dataset/case-02/plagiarized/L1/03 -> 1 files
/content/IR-Plag-Dataset (1)/IR-Plag-Dataset/case-02/plagiarized/L1/05 -> 1 files
/content/IR-Plag-Dataset (1)/IR-Plag-Dataset/case-02/plagiarized/L1/06 -> 1 files
/content/IR-Plag-Dataset (1)/IR-Plag-Dataset/case-02/plagiarized/L1/01 -> 1 files
/content/IR-Plag-Dataset (1)/IR-Plag-D

In [10]:
import pandas as pd

code_samples = []
for root, dirs, files in os.walk(extract_path):
    for file in files:
        if file.endswith(('.py', '.java', '.cpp')):
            try:
                with open(os.path.join(root, file), 'r', errors='ignore') as f:
                    code_samples.append(f.read())
            except Exception as e:
                print("Error reading", file, ":", e)

df_code = pd.DataFrame({'code': code_samples})
print(f" Loaded {len(df_code)} code files from all folders.")


 Loaded 467 code files from all folders.


In [11]:
import ast

class NormalizeAST(ast.NodeTransformer):
    def visit_Name(self, node):
        # Replace all variable names with a generic one
        return ast.copy_location(ast.Name(id="VAR", ctx=node.ctx), node)

    def visit_FunctionDef(self, node):
        # Replace function names with a generic one
        node.name = "FUNC"
        self.generic_visit(node)
        return node

    def visit_arg(self, node):
        # Replace argument names
        node.arg = "ARG"
        return node

def normalize_code(code):
    """
    Convert Python code into a normalized AST tree (ignoring variable names etc.)
    Returns a standardized string form for comparison.
    """
    try:
        tree = ast.parse(code)
        normalizer = NormalizeAST()
        normalized_tree = normalizer.visit(tree)
        return ast.dump(normalized_tree)
    except SyntaxError:
        # For non-Python code (e.g., Java, C++), just return the plain text
        return code


In [13]:
def detect_code_plagiarism(input_code, dataset, threshold=85):
    similarities = []
    for i, row in dataset.iterrows():
        score = plagiarism_score(input_code, row['code'])
        similarities.append(score)
        if i % 50 == 0:
            print(f"Compared {i+1}/{len(dataset)} files...")
    max_score = max(similarities)
    print(f"\n🔸 Highest similarity found: {max_score:.2f}%")
    if max_score >= threshold:
        return f" Code Plagiarized (Similarity: {max_score:.2f}%)"
    else:
        return f" Code is Original (Max Similarity: {max_score:.2f}%)"


In [15]:
from google.colab import files
print(" Upload your code file:")
uploaded_file = files.upload()

file_path = list(uploaded_file.keys())[0]

# Read your uploaded code
with open(file_path, 'r', errors='ignore') as f:
    user_code = f.read()

print(" Uploaded code loaded successfully!\n")

# Detect plagiarism
result = detect_code_plagiarism(user_code, df_code)
print("\nResult:", result)


 Upload your code file:


Saving input1.py to input1 (2).py
 Uploaded code loaded successfully!

Compared 1/467 files...
Compared 51/467 files...
Compared 101/467 files...
Compared 151/467 files...
Compared 201/467 files...
Compared 251/467 files...
Compared 301/467 files...
Compared 351/467 files...
Compared 401/467 files...
Compared 451/467 files...

🔸 Highest similarity found: 12.44%

Result: ✅ Code is Original (Max Similarity: 12.44%)
